# Determining th optimal number of hidden layers and neurons for an Artificial Neural Network(ANN)

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [12]:
data = pd.read_csv('/content/Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encode_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded =  onehot_encode_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encode_geo.get_feature_names_out(['Geography']))
data = data.drop(['Geography'], axis=1)
data = pd.concat([data, geo_encoded_df], axis=1)

In [13]:
X = data.drop(['Exited'], axis=1)
y = data['Exited']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)

with open('onehot_encode_geo.pkl', 'wb') as f:
    pickle.dump(onehot_encode_geo, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [16]:
## Define function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32, layers=1):
  model = Sequential()
  model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))

  for _ in range(layers-1):
    model.add(Dense(neurons, activation='relu'))

  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [ ]:
# Create a Keras Classifer, passing create_model as the build_fn and allowing params to be passed
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=10, verbose=0)

# Define the grid search parameters, updating the param_grid to reflect the change
param_grid = {
    'model__neurons': [16, 32, 64, 128], # access parameters of the model with 'model__' prefix
    'model__layers': [1, 2], # access parameters of the model with 'model__' prefix
    'epochs': [50, 100],
    'batch_size': [10, 32, 64] #add batch_size to grid search
}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))